In [1]:
import torch, numpy, json, os
from torch.utils.data import TensorDataset, DataLoader
from netdissect.progress import verbose_progress, default_progress
from netdissect.nethook import edit_layers
from netdissect.modelconfig import create_instrumented_model
from netdissect.zdataset import standard_z_sample, z_sample_for_model
from netdissect.easydict import EasyDict
from netdissect.aceoptimize import ace_loss
from netdissect.segmenter import UnifiedParsingSegmenter
from netdissect.fullablate import measure_full_ablation
from netdissect.plotutil import plot_tensor_images, plot_max_heatmap
import netdissect.aceoptimize
import netdissect.fullablate

verbose_progress(True)

layer = 'layer4'
dissectdir = 'dissect/churchoutdoor'
with open(os.path.join(dissectdir, 'dissect.json')) as f:
    dissection = EasyDict(json.load(f))

segmenter = UnifiedParsingSegmenter(segsizes=[256], segdiv='quad')
model = create_instrumented_model(dissection.settings)

classname = 'tree'
classnum = segmenter.get_label_and_category_names()[0].index((classname, 'object'))

edit_layers(model, [layer])

In [11]:
unit_count = 20

# then make a batch of 10 images
big_sample = z_sample_for_model(model, 1000, seed=3)
big_dataset = TensorDataset(big_sample)
big_loader = DataLoader(big_dataset, batch_size=10)



lrec = [l for l in dissection.layers if l.layer == layer][0]
rrec = [r for r in lrec.rankings if r.name == 'tree-iou'][0]
iou_scores = -torch.tensor(rrec.score)
iou_values, iou_order = (-iou_scores).sort(0)
iou_values = -iou_values
iou_ablation = torch.zeros_like(iou_scores)
iou_ablation[iou_order[:unit_count]] = 1

# load ablation from the tree model snapshot
snapdir = os.path.join(dissectdir, layer, 'ace_reg0.005', 'tree', 'snapshots')
data = torch.load(os.path.join(snapdir, 'epoch-9.pth'))
learned_scores = data['ablation'][0,:,0,0]
_, learned_order = (-learned_scores - iou_scores.cuda() * 1e-5).sort(0)
learned_values = learned_scores[learned_order]
learned_ablation = torch.zeros_like(learned_scores)
learned_ablation[learned_order[:unit_count]] = learned_scores[learned_order[:unit_count]]

progress = default_progress()

# (1) call ace_loss to get baseline tree pixels in batch
with torch.no_grad():
    baseline_loss = 0
    for [small_sample] in progress(big_loader):
        baseline_loss += ace_loss(segmenter, classnum, model, layer,
                torch.zeros_like(learned_scores)[None,:,None,None],
                torch.zeros_like(learned_scores)[None,:,None,None],
                small_sample, 0, 0, 0, run_backward=False,
                discrete_pixels=True,
                discrete_units=0,
                # mixed_units=True,
                ablation_only=True,
                fullimage_measurement=True,
                fullimage_ablation=True)

# (2) apply 20 unit iou ablation and call ace_loss to see the difference
with torch.no_grad():
    iou_loss = 0
    for [small_sample] in progress(big_loader):
        iou_loss += ace_loss(segmenter, classnum, model, layer,
                torch.zeros_like(iou_scores)[None,:,None,None].cuda(), # high_replacement
                iou_scores[None,:,None,None].cuda(),  # ablation
                small_sample, 0, 0, 0, run_backward=False,
                discrete_pixels=True,
                discrete_units=20,
                ablation_only=True,
                fullimage_measurement=True,
                fullimage_ablation=True)

# (3) apply 20 unit learned ablation and call ace_loss to see the difference
with torch.no_grad():
    learned_loss = 0
    for [small_sample] in progress(big_loader):
        learned_loss += ace_loss(segmenter, classnum, model, layer,
                torch.zeros_like(learned_scores)[None,:,None,None].cuda(),
                learned_scores[None,:,None,None].cuda(),
                small_sample, 0, 0, 0, run_backward=False,
                discrete_pixels=True,
                discrete_units=20,
                mixed_units=True,
                ablation_only=True,
                fullimage_measurement=True,
                fullimage_ablation=True)

print('iou', 1 - iou_loss / baseline_loss, 'learned', 1 - learned_loss / baseline_loss)
# There should be some ratio!!!



In [12]:
# (4) call measure_full_ablation on both the iou order to get the same raio
iou_measurements = measure_full_ablation(segmenter, big_loader, model, classnum, layer,
                                        iou_order[:unit_count], torch.ones_like(iou_values[:(unit_count)]))

# (5) and onthe learned ablation to get the same ratio!
learned_measurements = measure_full_ablation(segmenter, big_loader, model, classnum, layer,
                                        learned_order[:unit_count], learned_values[:(unit_count)])

print(iou_measurements)
print(learned_measurements)


In [13]:
print(1 - iou_measurements / iou_measurements[0], 1 - learned_measurements / learned_measurements[0])

In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot((1 - iou_measurements / iou_measurements[0]).numpy())
plt.plot((1 - learned_measurements / learned_measurements[0]).numpy())
plt.show()

In [ ]:
learned_measurements

In [ ]:
learned_measurements, baseline_loss  * 16 * 16, learned_loss * 16 * 16

In [ ]:
big_sample[0,:10]

In [ ]:
standard_z_sample(3, 512, seed=2)[0,:10]